# Cadeias de Markov em tempo contínuo

In [ ]:
import numpy as np
from IPython.display import Image, display

## Cracterização
Uma CMTC homogênea pode ser caracterizada de duas maneiras distintas:
* O processo sorteia uma variável aleatória com distribuição exponencial e taxa $\mu_{i}$. Tendo decorrido o intervalo de tempo sorteado, o estado de destino é definido através das probabilidades de transição $p_{ij}$
* O processo sorteia uma variável aleatória exponencial para cada destino com taxas $\mu_{ij} = \mu_{i} p_{ij}$, sendo selecionada a transição que corresponde ao menor intervalo de tempo.<br>

Equação de conversão da representação por probabilidades para representação por taxas:
* $\mu_{ij} = \mu_{i} p_{ij}$

Equação de conversão da representação por taxas para representação por probabilidades:
* $p_{ij}=\frac{\mu_{ij}}{\mu_{ij}+ ...+\mu_{ik}}$

### Conversão de representação
Uma cadeia de Markov pode ser representada pela sua matriz de taxas de transição (Q), ou pelo vetor de taxa de permanência (Mu) e matriz de probabilidades de trandição (P). Tanto para o cálculo do regime do regime transitório quanto para o cálculo estacionário precisamos da matriz Q.<br>
Com base na propriedade apresentada acima podemos usar o seguite algoritmo para converter a representação pelo Mu e matriz P para a representação pela matriz Q, incluido as transições para o próprio estado.
* Multiplicar cada valor do vetor Mu pela linha correpondente em P
* Os elementos da diagonal principal devem ser iguais a soma dos demais valores da linha com sinal negativo (somar todos os elemento, trocar o sinal e somar novamente o elemento da diagonal principal.

## Exemplo 1
A função **cmtcPtoQ** converte a represnetação por probabilidade (P) para a representação por taxas (Q). Recebe como argumentos a matriz de transição de um passo (P) e o vetor de taxas de permanência (Mu) e retorna a matriz Q.

In [ ]:
def cmtcPtoQ(P, Mu):
    [r,c] = P.shape
    Q = np.empty_like(P)
    if ((r != c) | np.any(np.sum(P, 1) != 1)):
        raise Exception('Matriz P invalida!')

    for x in range(0, r):
        Q[x, :] = Mu[x] * P[x, :]
        Q[x, x] = -np.sum(Q[x, :])
    return Q

# Matriz P do exemplo 1
P = np.array([[0, 1/3, 2/3, 0],[5/6, 0, 0, 1/6],[10/11, 0, 0, 1/11], [0, 1/2, 1/2, 0]], 
             dtype=np.float64)

# Vetor de taxas de permanência
Mu = np.array([30, 120, 110, 200], dtype=np.float64)

# Imprime matriz Q
print(cmtcPtoQ(P,Mu))

## Regime transitório
Toda CMTC possui uma matriz denominada matriz geradora infinitesimal $\mathbf{Q}$, onde $q_{i,j}$ é a taxa de transição do estado $i$ para o estado $j$.Os elementos da diagonal $q_{i,i}$ são escolhidos, por definição, iguais ao oposto (negativo) da soma dos demais elementos da mesma linha $i$:<br>
* $q_{i,j} = \mu _{i,j}$   &nbsp;&nbsp;   $\forall i\neq j$
* $q_{i,i} = -\sum_{i\neq i}^{}\mu_{i,j} = -\mu _{i}$<br><br>
$\mathbf{Q} = \begin{bmatrix} 
-\mu_{1}&  \mu_{1,2}&  \cdots & \mu_{1,n}\\ 
\mu_{2,1}&  -\mu_{2}&  \cdots & \mu_{2,n}\\ 
\vdots &  \vdots & \ddots  & \vdots\\
\mu_{n,1}&  \mu_{n,2} &  \cdots & -\mu_{n}
\end{bmatrix}$<br><br>
O regime transitório consiste em determinar o vetor $\boldsymbol{ \pi } \left ( t \right )$ das probabilidades de estado $\pi _{j}\left ( t \right ) = P\left [ X\left ( t \right ) \right ]=j, j\in \left \{ estados \right \}$, em todos os instantes do processo:<br><br>
$\boldsymbol{\pi }(t)=\begin{bmatrix} \boldsymbol{} \pi _{1}(t)&  \pi _{2}(t)& \cdots &  \pi _{n}(t)\end{bmatrix}$<br><br><br>
O cálculo de $\boldsymbol{\pi }(t)$ é realizado pela seguinte equação:<br><br>
$\boldsymbol{\pi }(t) = \boldsymbol{\pi }(0) e^{\mathbf{Q} t}$<br>
onde<br>
$\boldsymbol{\pi }(0)=\begin{bmatrix} \boldsymbol{} \pi _{1}(0)&  \pi _{0}(t)& \cdots &  \pi _{n}(0)\end{bmatrix}$<br><br><br>
Calculamos $e^{\mathbf{Q}t}$ utlizando a equação do limite fundamental da exponencial:<br><br>
$e^{\mathbf{Q}t} = \lim_{n \to \infty}\left ( \mathbf{I} + \frac{\mathbf{Q}t}{n}\right )^{n}
$<br>
escolhendo um valor de n suficientemente grande de modo que tenha números positivos menores do que 1 em todas as posições

In [ ]:
def cmtc_eQt(Q, t, n_max):
    I = np.identity(Q.shape[1])
    for n in range(100, n_max, 100):
        X = np.linalg.matrix_power(I + (t * Q / n), n)
        if np.all(X < 1):
            break
    return X

## Exemplo 3
$\mathbf{Q} = \begin{bmatrix} 
-30 & 10 & 20 & 0 \\ 100 & -120 & 0 & 20 \\ 100 &  0 & -110  & 10 \\ 0 & 100 & 100 & 200
\end{bmatrix}$

In [ ]:
Q = np.array([[-30, 10, 20, 0],[100, -120, 0, 20], [100, 0, -110, 10],[0,100, 100, -200]],
             dtype=np.float64)
PI0 = np.array([1, 0, 0, 0], dtype=np.float64)
t = 0.0035
n_max = 2000
eQt = cmtc_eQt(Q, t, n_max)

print(eQt)
print(np.dot(PI0, eQt))

## Exemplo 3
Calcular as probabilidades no regime permanente da CMTC representada pela seguinte matriz geradora:

$\mathbf{Q }= \begin{bmatrix} 
-1/12 & 1/12 & 0 & 0 & 0 & 0 & 0 & 0   \\ 1/2 & -5 & 3 & 3/2 & 0 & 0 & 0 & 0 \\ 
0 & 0 & -3/2 & 0 & 3/2 & 0 & 0  & 0    \\ 0 & 0 & 0 & -3 & 3 & 0 & 0 & 0     \\ 
0 & 0 & 0 & 0 & -1/6 & 1/15 & 1/10 & 0 \\ 3 & 0 & 0 & 0 & 0 & -3 & 0 & 0     \\
0 & 0 & 0 & 0 & 0 & 0 & -4 & 4         \\ 3/2 & 0 & 0 & 0 & 0 & 0 & 0 & -3/2 \\
\end{bmatrix}$

&nbsp; Equações de quilibrio:<br>
&nbsp;&nbsp; $1/12\pi_{1} = 1/2\pi_{2} + 3\pi_{6} + 3/2\pi_{8}$<br>
&nbsp;&nbsp; $\left ( 1/2 + 3 + 3/2 \right )\pi_{2} = 1/12\pi_{1}$<br>
&nbsp;&nbsp; $3/2\pi_{3} = 3\pi_{2}$<br>
&nbsp;&nbsp; $3\pi_{4} = 3/2\pi_{2}$<br>
&nbsp;&nbsp; $\left ( 1/15 + 1/10 \right )\pi_{5} = 3/2\pi_{3} + 3\pi_{4}$<br>
&nbsp;&nbsp; $3\pi_{6} = 1/15\pi_{5}$<br>
&nbsp;&nbsp; $4\pi_{7} = 1/10\pi_{5}$<br>
&nbsp;&nbsp; $3/2\pi_{8} = 4\pi_{7}$<br>
&nbsp;&nbsp; $\pi_{1} + \pi_{2} + \pi_{3} + \pi_{4} + \pi_{5} + \pi_{6} + \pi_{7} + \pi_{8}= 1$<br>


Escrevendo o sistema na foma matricial temos:<br><br>
&nbsp;&nbsp; 
$\mathbf{A}= \begin{bmatrix} 
-1/12 & 1/12 & 0 & 0 & 0 & 0 & 0 & 3/2  \\ 1/2 & -5 & 0 & 0 & 0 & 0 & 0 & 0 \\ 
0 & 3 & -3/2 & 0 & 0 & 0 & 0 & 0 \\ 0 & 3/2 & 0 & -3 & 0 & 0 & 0 & 0     \\ 
0 & 0 & 3/2 & 3 & -1/6 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1/15 & -3 & 0 & 0     \\
0 & 0 & 0 & 0 & 1/10 & 0 & -4 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 4 & -3/2 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
\end{bmatrix}$


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
$\mathbf{B} =\begin{bmatrix}0\\0\\0\\0\\0\\0\\0\\0\\1\end{bmatrix}$
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
$\mathbf{\pi } =\begin{bmatrix}
\pi_{1}\\ \pi_{2}\\ \pi_{3}\\ \pi_{4}\\ \pi_{5}\\ \pi_{6}\\ \pi_{7}\\ \pi_{8}
\end{bmatrix}$<br><br>
&nbsp;&nbsp;
$\mathbf{\pi } = \mathbf{A}^{+}\cdot\mathbf{B}$
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$\mathbf{\pi } =\begin{bmatrix}
0.6412\\0.0107\\0.0214\\0.0053\\0.2885\\0.0064\\0.0072\\0.0192\end{bmatrix}$<br>

* A pseudo inversa de uma matriz $\mathbf{A}$, denotada por $\mathbf{A}^{+}$, é a mais importante generalização da inversa de uma matriz. 
* $\mathbf{A}^{+}$ é usada para calcular uma solução de "melhor ajuste" (pelo método dos mínimos quadrados) para um sistema de equações lineares.

In [ ]:
A = np.array([[-1/12, 1/2, 0, 0, 0, 3, 0, 3/2], [1/12, -5, 0, 0, 0, 0, 0, 0], 
              [0, 3, -3/2, 0, 0, 0, 0, 0], [0, 3/2, 0, -3, 0, 0, 0, 0],
              [0, 0, 3/2, 3, -1/6, 0, 0, 0], [0, 0, 0, 0, 1/15, -3, 0, 0],
              [0, 0, 0, 0, 1/10, 0, -4, 0], [0, 0, 0, 0, 0, 0, 4, -3/2],
              [1, 1, 1, 1, 1, 1, 1, 1]], 
             dtype=np.float64)
B= np.array([0, 0, 0, 0, 0, 0, 0, 0, 1] , dtype=np.float64)

A_pinv = np.linalg.pinv(A)
PI = np.dot(A_pinv,B)
print(PI)

## Transições para o mesmo estado
Quando utilizamos a representação da CMTC com as taxas de transição, pode-se suprimir os enlaces para o mesmo estado sem alterar o comportamento da cadeia.<br><br>

In [ ]:
display(Image('markov1.png'))